# Alpha-based crypto quant trading strategy

## Setup

Download data or load from disk.

In [ ]:
import os

import vectorbt as vbt

timeframe = "1d"
start_date = "2023-01-01"
end_date = "2024-12-31"
symbols = [
    "SOL/USDT:USDT",
    "ETH/USDT:USDT",
    "BTC/USDT:USDT",
    "XRP/USDT:USDT",
    "ZEC/USDT:USDT",
    "DOGE/USDT:USDT",
    "DASH/USDT:USDT",
    "LTC/USDT:USDT",
    "ADA/USDT:USDT",
    "AVAX/USDT:USDT",
    "TRUMP/USDT:USDT",
    "GIGGLE/USDT:USDT",
    "LINK/USDT:USDT",
    "SUI/USDT:USDT",
]

data_path = f"crypto_data_{timeframe}_{start_date}_{end_date}.pkl"

if os.path.exists(data_path):
    data = vbt.CCXTData.load(data_path)
else:
    data = vbt.CCXTData.download(symbols, timeframe=timeframe, start=start_date, end=end_date)
    data.save(data_path)

Generate trading orders.

In [ ]:
leverage = 10
balance = 60
stake_amount = balance * leverage
init_cash = stake_amount / len(symbols)
fees = 0.0005

# Alpha#1: (rank(Ts_ArgMax(SignedPower(((returns < 0) ? stddev(returns, 20) : close), 2.), 5)) - 0.5)
close = data.get("Close")
returns = close.pct_change(fill_method=None)
stddev_20 = returns.rolling(20).std()
condition = returns < 0
signed_power = (stddev_20.where(condition, close)) ** 2
argmax_5 = signed_power.rolling(5).apply(lambda x: x.argmax(), raw=True)
ranked = argmax_5.rank(axis=1, pct=True) - 0.5
cs_rank = ranked.rank(axis=1, ascending=False, method="first")
entries = cs_rank <= 2
exits = cs_rank > 2
pf = vbt.Portfolio.from_signals(close, entries, exits, init_cash=init_cash, fees=fees)

In [ ]:
total_profit = pf.total_profit().sum()
balance_sum = balance + total_profit
print(f"Profit sum across all symbols: {total_profit}")
print(f"Balance sum across all symbols: {balance_sum}")
pf.total_profit()